# Introduction
As a final step in fixing the biomass reaction, we will need to re-fit the growth associated maintanence (GAM) in the reaction to capture the amount of energy needed for the production of biomass. Beata originally already did this in her thesis, but as we've changed the biomass reaction quite a bit in the mean time we need to fit this again.

To do so, we will use the chemostat data presented in her thesis in figure 4.5. With this data we can minimize the error of the linear regression to estimate the best possible GAM that captures our data and model.

In [33]:
import cameo
import pandas as pd
import cobra.io
import escher

In [34]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

## NGAM
Before we fit the GAM, we need to make sure the NGAM is still correct, as we've changed quite a bit in the model since we did this last time. Based on Beata's thesis, we know the intercept of figure 4.5, i.e. the amount of glucose consumed when the cell is not growing. This is then the sugar that is converted into energy which is used for maintanence. 

To estimate the NGAM, we will fix the glucose consumption rate to that measured at a growth rate of 0. We will then block biomass formation. We can set the objective of the model to maximize ATP maintenance. The flux through the ATPM reaction then should become the lower bound of this reaction as it represents the NGAM. 

In [3]:
with model:
    model.reactions.EX_glc__D_e.bounds = (-0.1551,0) #0.1551 mmol glucose/gCDW/h is consumed at a growth rate of 0
    model.reactions.biomass.bounds = (0,0)
    model.reactions.ATPM.bounds = (0,1000) #remove the original estimated NGAM that was given
    model.objective = 'ATPM'
    print(model.optimize())

<Solution 3.141 at 0x1b02260ba48>


so the non-growth associated maintenance should be fixed at 3.141 mmolATP/gCDW/h. This will be set by fixing the lower bounds of the ATPM reaction to this flux t oforce this flux through, regardless of the growth rate.

In [4]:
model.reactions.ATPM.bounds = (3.141, 1000)

In [5]:
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

## GAM
Now that we had fixed and fit the NGAM, we can move forward and fit the GAM to the model. 

For now I had to estimate the data from the image in Beata's thesis... This is not ideal. Martyn would look to see if he could find anything in the lab when they are allowed back in. 

In [8]:
#import data
df = pd.read_csv('../../databases/Growth data.csv', sep = ';')
#should be in the long format

In [9]:
del df['Unnamed: 6']

In [10]:
del df['1 mm']
del df['corresponds to']
del df['0.074074074']
del df['mmol/gcdw/h']

In [11]:
df

,Growth rate,Glucose_consumption_1,Glucose_consumption_2,Glucose_consumption_3,Mean_glucose_consumption,st_dev
0,0.05,1.000000,1.148148,1.259259,1.135802,0.130070
1,0.10,1.703704,1.851852,2.014815,1.856790,0.155614
2,0.20,3.000000,3.111111,3.200000,3.103704,0.100206
3,0.40,5.925926,6.814815,7.962963,6.901235,1.021265


In [37]:
#add a function that will fit the gam
#GAM_list = list of GAMS to test, e.g. from 10 to 100 with an interval of 1. This will be changed with each iteration
#df = Data frame with each condition and the mean and stdev measured glucose consumption rates
def iteration(model, GAM_list, df):
    R_list = []
    for GAM in GAM_list:
        model_copy = copy(model) 
        model_copy.reactions.biomass.add_metabolites({
            model_copy.metabolites.atp_c: -GAM,
            model_copy.metabolites.adp_c: GAM,
            model_copy.metabolites.pi_c:GAM,
            model_copy.metabolites.h2o_c: -GAM,
            model_copy.metabolites.h_c: GAM}) 
        R = []
        for index,row in df.iterrows():
            fx_biomass = row['Growth rate']    #here call the condition set by the data frame in the above
            model_copy.reactions.biomass.bounds = (fx_biomass,fx_biomass)
            model_copy.reactions.biomass.objective_coefficient = 0 #remove biomass objective
            model_copy.reactions.EX_glc__D_e.objective_coefficient = 1 #minimize the Glucose uptake as objective
            vG = model_copy.optimize().objective_value #predictied glucose consumption to get the fixed growth rate for this GAM
            meanEXP = row['Mean_glucose_consumption'] #mean measured glucose consumption rate for this growth rate 
            stdEXP = row['st_dev']#stdev at this growth rate
            R.append(abs(vG - meanEXP)/stdEXP) #calculate how far the predicted vG differs from the measured and store it in a list
        R_list.append(sum(R))
    #find the minimum from the list of GAMs tried
    smallest_R = min(R_list)
    #then find the GAM that associates to the smallest R
    find_index = R_list.index(smallest_R)
    best_GAM = GAM_list[find_index]
    return print(best_GAM, R_list)

In [38]:
#first remove the GAM from the model biomass reaction to start with a clean slate
model.reactions.biomass.add_metabolites({
    model.metabolites.atp_c: 104.9974,
    model.metabolites.adp_c: -104.9974,
    model.metabolites.pi_c:-104.9974,
    model.metabolites.h2o_c: 104.9974,
    model.metabolites.h_c: -104.9974
})

In [39]:
#Allow any glucose uptake
model.reactions.EX_glc__D_e.bounds = (-1000,0)

In [40]:
#1st iteration, going from 10 to 150, in intervals of 10
GAM_list = list(range(10,151,10))

In [41]:
iteration(model, GAM_list, df)

10 [88.9105103451747, 92.20964351619276, 97.16436267622984, 103.81121955183708, 112.24216662863662, 122.35947638235238, 134.16300442835472, 147.65275076664352, 162.82871539721754, 179.6908983200775, 198.23929953522205, 218.473919042655, 240.3947568423735, 264.00181293437737, 289.29508731866764]


In [28]:
#still get cumulative GAM in the model...

In [42]:
model.reactions.biomass.metabolites

{<Metabolite pydx5p_c at 0x1c67c394b48>: -0.000199107142857143,
 <Metabolite adocbl_c at 0x1c67cc01208>: -0.000199107142857143,
 <Metabolite qh2_c at 0x1c67cc79788>: -0.000267857142857145,
 <Metabolite fru_c at 0x1c67cc8b488>: -0.104791964285714,
 <Metabolite gdpmann_c at 0x1c67cc8b208>: -0.00572410714285714,
 <Metabolite coa_c at 0x1c67be2a0c8>: -0.000150000000000001,
 <Metabolite leu__L_c at 0x1c67cb53688>: -0.347473214285714,
 <Metabolite ptrc_c at 0x1c67cb1ff88>: -0.00535714285714286,
 <Metabolite his__L_c at 0x1c67cb1a188>: -0.0778214285714286,
 <Metabolite gmp_c at 0x1c67cae6248>: -0.0914575678571428,
 <Metabolite ppi_c at 0x1c67cae1708>: -0.00107142857142857,
 <Metabolite pro__L_c at 0x1c67cacda88>: -0.166535714285714,
 <Metabolite asn__L_c at 0x1c67cac00c8>: -0.221941964285714,
 <Metabolite fad_c at 0x1c67ca68708>: -0.000199107142857144,
 <Metabolite val__L_c at 0x1c67ca658c8>: -0.376142857142857,
 <Metabolite thr__L_c at 0x1c67ca4fc08>: -0.268767857142857,
 <Metabolite phe__L_

In [ ]:
R_best = iteration(model, 10...100)
R_best = iteration(model, R_best-10...1...R_best+10)
R_best = iteration(model, R_best-1...0.1...R_best+1)

need chemostate data:
dilution rate vs consumption rates

if we make more changes o the model: when/if to refit?